In [12]:
!pip install SPARQLWrapper

In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

query = """
SELECT ?artist ?genre
WHERE {
  ?artist rdf:type dbo:MusicalArtist .
  ?artist dbo:genre ?genre .
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

artists = []
genres = []

for result in results["results"]["bindings"]:
    artist_name = result["artist"]["value"].split("/")[-1]
    genre_name = result["genre"]["value"].split("/")[-1]
    artists.append(artist_name)
    genres.append(genre_name)


In [14]:
!pip install rdflib


In [15]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL

MUSIC = Namespace("http://example.org/music#")

g = Graph()

g.bind("music", MUSIC)

# Define classes
g.add((MUSIC.MusicalArtist, RDF.type, OWL.Class))
g.add((MUSIC.Genre, RDF.type, OWL.Class))
g.add((MUSIC.Song, RDF.type, OWL.Class))

# Define properties
g.add((MUSIC.hasGenre, RDF.type, OWL.ObjectProperty))
g.add((MUSIC.hasArtist, RDF.type, OWL.ObjectProperty))
g.add((MUSIC.hasSong, RDF.type, OWL.ObjectProperty))

# Define relationships
g.add((MUSIC.hasGenre, RDFS.domain, MUSIC.MusicalArtist))
g.add((MUSIC.hasGenre, RDFS.range, MUSIC.Genre))
g.add((MUSIC.hasArtist, RDFS.domain, MUSIC.Song))
g.add((MUSIC.hasArtist, RDFS.range, MUSIC.MusicalArtist))
g.add((MUSIC.hasSong, RDFS.domain, MUSIC.Genre))
g.add((MUSIC.hasSong, RDFS.range, MUSIC.Song))

# Save ontology to file
g.serialize("music.owl", format="xml")


<Graph identifier=N7e1385301c3243afa79055dfd5fa462f (<class 'rdflib.graph.Graph'>)>

In [16]:
!pip install spacy
!python -m spacy download en_core_web_sm


2023-04-17 18:05:10.858010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-17 18:05:10.858060: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-17 18:05:14.304538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2023-04-17 18:05:14.304568: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-17 18:05:14.307778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-15CCF22
2023-04-17 18:05:14.307858: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-15CCF22



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
import spacy

nlp = spacy.load("en_core_web_sm")

artist_tokens = []
genre_tokens = []

for artist in artists:
    doc = nlp(artist)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    artist_tokens.append(tokens)

for genre in genres:
    doc = nlp(genre)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    genre_tokens.append(tokens)


In [18]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal

MUSIC = Namespace("http://example.org/music#")

g = Graph()

g.bind("music", MUSIC)

# Load ontology from file
g.parse("music.owl")

# Add instances to the graph
for i, artist in enumerate(artist_tokens):
    artist_name = "artist{}".format(i)
    artist_uri = MUSIC[artist_name]
    g.add((artist_uri, RDF.type, MUSIC.MusicalArtist))
    for token in artist:
        g.add((artist_uri, RDFS.label, Literal(token)))

for i, genre in enumerate(genre_tokens):
    genre_name = "genre{}".format(i)
    genre_uri = MUSIC[genre_name]
    g.add((genre_uri, RDF.type, MUSIC.Genre))
    for token in genre:
        g.add((genre_uri, RDFS.label, Literal(token)))

# Add relationships to the graph
for i, artist in enumerate(artist_tokens):
    artist_name = "artist{}".format(i)
    artist_uri = MUSIC[artist_name]
    for j, genre in enumerate(genre_tokens):
        genre_name = "genre{}".format(j)
        genre_uri = MUSIC[genre_name]
        if set(artist).intersection(set(genre)):
            g.add((artist_uri, MUSIC.hasGenre, genre_uri))

# Save the graph to a file
g.serialize("music.rdf", format="xml")


<Graph identifier=N97594efa450a44279b6022ede45e631b (<class 'rdflib.graph.Graph'>)>

In [19]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

def get_recommendations(artist_name):
    query = f"""
    SELECT ?song
    WHERE {{
      ?artist rdf:type dbo:MusicalArtist .
      ?artist dbo:genre ?genre .
      ?song dbo:musicalArtist ?artist .
      FILTER (CONTAINS(LCASE(?artistName), "{artist_name.lower()}"))
    }}
    """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()

    recommendations = []

    for result in results["results"]["bindings"]:
        song_name = result["song"]["value"].split("/")[-1]
        recommendations.append(song_name)

    return recommendations


In [23]:
get_recommendations("Lata Mangeshkar")

[]